# Fine-tuning

Test several pre-trained models from HuggingFace, fine tune on adus data, and hypertune parameters

In [ ]:
!pip install pandas
!pip install datasets
!pip install transformers
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install "ray[tune]" optuna

## Preparing our Data

In this notebook, we'll start by using a local dataset (instead of using a dataset stored at Hugging Face).
Let's load data for our classification task.

### Loading dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# Importing the dataset
dataset = pd.read_excel('/content/drive/Shareddrives/PLN/Assignment 2/data/OpArticles_ADUs.xlsx')
dataset = dataset.drop(columns=['article_id', 'annotator', 'node','ranges'])
dataset['label'].replace(['Value', 'Value(+)', 'Value(-)', 'Fact', 'Policy'],[0,1,2,3,4], inplace=True)

print(dataset.info())
print(dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16743 entries, 0 to 16742
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  16743 non-null  object
 1   label   16743 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 261.7+ KB
None
                                              tokens  label
0           O facto não é apenas fruto da ignorância      0
1  havia no seu humor mais jornalismo (mais inves...      0
2                              É tudo cómico na FIFA      0
3  o que todos nós permitimos que esta organizaçã...      0
4            não nos fazem rir à custa dos poderosos      0


For ease of usage with Transformer models, we convert the dataset into a Hugging Face dataset and split it into train, validation and test sets.

In [ ]:
from datasets import Dataset

dataset_hf = Dataset.from_pandas(dataset)

In [ ]:
from datasets import DatasetDict

# 90% train, 10% test+validation
train_test = dataset_hf.train_test_split(test_size=0.1, shuffle=True, seed=42)

# Split the 10% test+validation set in half test, half validation
valid_test = train_test['test'].train_test_split(test_size=0.5, shuffle=True, seed=42)

# gather everyone if you want to have a single DatasetDict
train_valid_test_dataset = DatasetDict({
    'train': train_test['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

In [ ]:
train_valid_test_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'label'],
        num_rows: 15068
    })
    validation: Dataset({
        features: ['tokens', 'label'],
        num_rows: 837
    })
    test: Dataset({
        features: ['tokens', 'label'],
        num_rows: 838
    })
})

## Fine-tuning a pretrained model

### Tokenizer

We first load the tokenizer for our model:

In [ ]:
from transformers import AutoTokenizer

def get_tokenizer(name):
    return AutoTokenizer.from_pretrained(name)

Now we need to [preprocess](https://huggingface.co/docs/transformers/preprocessing) our data.

Obtaining the length of the longest sequences in our data splits

In [ ]:
def find_max_length(dataset):
    return len(max(dataset, key=lambda x: len(x.split())).split())

train_max_length = find_max_length(train_valid_test_dataset["train"]["tokens"])
val_max_length = find_max_length(train_valid_test_dataset["validation"]["tokens"])
test_max_length = find_max_length(train_valid_test_dataset["test"]["tokens"])

print(f"Longest sequence in train set has {train_max_length} words")
print(f"Longest sequence in val set has {val_max_length} words")
print(f"Longest sequence in test set has {test_max_length} words")

Longest sequence in train set has 81 words
Longest sequence in val set has 51 words
Longest sequence in test set has 59 words


Tokenize entire dataset

In [ ]:
# Define tokenizer
tokenizer = None

def tokenize_dataset(sample):
    return tokenizer(sample["tokens"], truncation=True, max_length=81, padding="max_length")

def get_tokenized_data(dataset):
    return dataset.map(tokenize_dataset, batched=True, remove_columns=["tokens"])

### Loading the model

Since we want to use the model for classification, we should load it with an appropriate classification head:

In [ ]:
from transformers import AutoModelForSequenceClassification

# Define later
model_name = None

def get_model():
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True)
    model.cuda()

    return model

### Fine-tuning

The next step is to [fine-tune](https://huggingface.co/docs/transformers/training) the model with our train data. To do so, we can make use of a [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer).
There are several aspects of training that you can specify via [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments).

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from IPython.display import display

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def get_trainingArgs():
    return TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        fp16=True,
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        seed=42,
        data_seed=42,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1"
    )

def get_trainer(model_init_, args_, dataset_, tokenizer_, data_collator_, compute_metrics_):
    return Trainer(
        model_init=model_init_,
        args=args_,
        train_dataset=dataset_["train"],
        eval_dataset=dataset_["validation"],
        tokenizer=tokenizer_,
        data_collator=data_collator_,
        compute_metrics=compute_metrics_
    )

def train_model(model_name):
  global tokenizer
  tokenizer = get_tokenizer(model_name)
  tokenized_dataset = get_tokenized_data(train_valid_test_dataset)

  trainer = get_trainer(
    get_model,
    get_trainingArgs(),
    tokenized_dataset,
    tokenizer,
    DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics
    )
  
  # Train Model
  display(trainer.train())

  # Check performance in validation set
  display(trainer.evaluate())

  # Check how the model fares in our test set.
  display(trainer.predict(test_dataset=tokenized_dataset["test"]))

  # Save model for future use
  trainer.save_model('/content/drive/Shareddrives/PLN/Assignment 2/models/baseline/' + model_name)


#### Custom training to use a weighted loss

Useful for our unbalanced training set

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_valid_test_dataset['train']['label']),
    y=train_valid_test_dataset['train']['label']
    )
class_weights

array([0.41220079, 2.36732129, 1.15066819, 0.92356727, 5.00598007])

In [ ]:
from torch import nn, tensor

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        #inputs = inputs.to(device)
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (5 labels with different weight)
        loss_fct = nn.CrossEntropyLoss(weight=tensor([0.41, 2.37, 1.15, 0.92, 5.01]))
        loss_fct.cuda()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def get_trainer_custom(model_init_, args_, dataset_, tokenizer_, data_collator_, compute_metrics_):
    return CustomTrainer(
        model_init=model_init_,
        args=args_,
        train_dataset=dataset_["train"],
        eval_dataset=dataset_["validation"],
        tokenizer=tokenizer_,
        data_collator=data_collator_,
        compute_metrics=compute_metrics_
    )

def train_model_custom(model_name):
  global tokenizer
  tokenizer = get_tokenizer(model_name)
  tokenized_dataset = get_tokenized_data(train_valid_test_dataset)

  trainer = get_trainer_custom(
    get_model,
    get_trainingArgs(),
    tokenized_dataset,
    tokenizer,
    DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics
    )
  
  # Train Model
  display(trainer.train())

  # Check performance in validation set
  display(trainer.evaluate())

  # Check how the model fares in our test set.
  display(trainer.predict(test_dataset=tokenized_dataset["test"]))

  # Save model for future use
  trainer.save_model('/content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/' + model_name)

#### Hyperparameter Tuning

In [ ]:
def my_hp_space(trial):
  return {
    "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
    "per_device_eval_batch_size": trial.suggest_categorical("per_device_eval_batch_size", [16, 32]),
    "weight_decay": trial.suggest_categorical("weight_decay", [0, 0.01, 0.1]),
    "learning_rate": trial.suggest_categorical("learning_rate", [2e-6, 2e-5, 2e-4]),
    "num_train_epochs": trial.suggest_categorical("num_train_epochs", [3, 5])
  }

def my_objective(metrics):
  return metrics["eval_f1"]

def get_trainer_hyper(model_init_, args_, dataset_, tokenizer_, data_collator_, compute_metrics_):
  return Trainer(
      model_init=model_init_,
      args=args_,
      train_dataset=dataset_["train"],
      eval_dataset=dataset_["validation"],
      tokenizer=tokenizer_,
      data_collator=data_collator_,
      compute_metrics=compute_metrics_
  )

def train_model_hyper(model_name):
  global tokenizer
  tokenizer = get_tokenizer(model_name)
  tokenized_dataset = get_tokenized_data(train_valid_test_dataset)

  trainer = get_trainer_hyper(
    get_model,
    get_trainingArgs(),
    tokenized_dataset,
    tokenizer,
    DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics
    )
  
  best_run = trainer.hyperparameter_search(
      hp_space=my_hp_space,
      n_trials=5,
      direction="maximize",
      compute_objective=my_objective
  )

  print("***** Hypertuning best run *****")
  display(best_run)
  
  # Set params
  for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

  # Save model for future use
  trainer.save_model('/content/drive/Shareddrives/PLN/Assignment 2/models/baseline/hyper/' + model_name)

### PT - [Geotrend/distilbert-base-pt-cased](https://huggingface.co/Geotrend/distilbert-base-pt-cased)

In [ ]:
model_name = "Geotrend/distilbert-base-pt-cased"

In [ ]:
train_model(model_name)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/Geotrend/distilbert-base-pt-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/81477275f67d8ef9f4af68aeb6799256587efc91da21e00c93a447c4f529447f.6168459fbc8e0022c73def07f9e7976d25cbac85c752238f63549206aeba6abc
Model config DistilBertConfig {
  "_name_or_path": "Geotrend/distilbert-base-pt-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinuso

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.091420,0.526882,0.401252,0.503721,0.376302
2,1.161900,1.080899,0.554361,0.493484,0.555143,0.478184
3,0.958200,1.091655,0.556750,0.508679,0.541024,0.498284


...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.5567502986857825,
 'eval_f1': 0.5086788931624167,
 'eval_loss': 1.0916552543640137,
 'eval_precision': 0.5410243881818309,
 'eval_recall': 0.4982836960672012,
 'eval_runtime': 0.9421,
 'eval_samples_per_second': 888.455,
 'eval_steps_per_second': 28.66}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 0.8022, -0.5796,  1.922 , -0.1488, -2.469 ],
       [ 2.906 , -1.33  ,  0.197 ,  0.2345, -2.705 ],
       [ 2.113 , -1.627 ,  1.341 ,  0.3484, -2.807 ],
       ...,
       [ 2.594 , -1.692 , -0.6704,  1.192 , -1.873 ],
       [ 0.4778, -1.39  ,  2.557 , -0.1533, -2.01  ],
       [ 2.281 , -1.909 ,  1.6   , -0.2612, -2.05  ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 2,
       0, 1, 0, 2, 3, 3,

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/Geotrend/distilbert-base-pt-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/81477275f67d8ef9f4af68aeb6799256587efc91da21e00c93a447c4f529447f.6168459fbc8e0022c73def07f9e7976d25cbac85c752238f63549206aeba6abc
Model config DistilBertConfig {
  "_name_or_path": "Geotrend/distilbert-base-pt-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.2",
  "vocab_size": 25000
}

loading file https://huggingface.co/Geotrend/distilbert-base-pt-cased/resolve/main/vocab.txt from cache at /root/.cache/h

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/Geotrend/distilbert-base-pt-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/81477275f67d8ef9f4af68aeb6799256587efc91da21e00c93a447c4f529447f.6168459fbc8e0022c73def07f9e7976d25cbac85c752238f63549206aeba6abc
Model config DistilBertConfig {
  "_name_or_path": "Geotrend/distilbert-base-pt-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {


...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.48984468339307047,
 'eval_f1': 0.48395577705005427,
 'eval_loss': 1.1156558990478516,
 'eval_precision': 0.45271520149199035,
 'eval_recall': 0.569497627229586,
 'eval_runtime': 0.9517,
 'eval_samples_per_second': 879.488,
 'eval_steps_per_second': 28.371}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 0.1675 , -0.8647 ,  2.055  , -0.1212 , -1.754  ],
       [ 1.467  , -0.874  ,  0.6045 ,  0.2031 , -1.951  ],
       [ 1.034  , -1.188  ,  1.406  ,  0.3562 , -2.14   ],
       ...,
       [ 1.409  , -1.322  , -0.05206,  1.258  , -1.819  ],
       [ 0.4797 , -1.363  ,  2.254  , -0.3884 , -1.489  ],
       [ 1.091  , -1.681  ,  1.879  ,  0.0966 , -1.884  ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

### PT - [Geotrend/bert-base-pt-cased](https://huggingface.co/Geotrend/bert-base-pt-cased)

In [ ]:
model_name = "Geotrend/bert-base-pt-cased"

In [ ]:
train_model(model_name)

https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp0rcghb61


Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

storing https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/29ca3cacb025b278614e903a0eca5e20b79457c33dcf2fa64250874c94b5a2c7.25d8d06fb0679146a3ed2a3463e3585380bff882fe6e1ebc497196e40dbbd7fa
creating metadata file for /root/.cache/huggingface/transformers/29ca3cacb025b278614e903a0eca5e20b79457c33dcf2fa64250874c94b5a2c7.25d8d06fb0679146a3ed2a3463e3585380bff882fe6e1ebc497196e40dbbd7fa
https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpjhgcq87k


Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

storing https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/0ebe421d721676677f14dd2556297b76144988966ccb9ca2655253e71be546e1.49ec507c8c81787c05598347db28fe699ccea17f6f63e025aecca400cf4922f8
creating metadata file for /root/.cache/huggingface/transformers/0ebe421d721676677f14dd2556297b76144988966ccb9ca2655253e71be546e1.49ec507c8c81787c05598347db28fe699ccea17f6f63e025aecca400cf4922f8
loading configuration file https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0ebe421d721676677f14dd2556297b76144988966ccb9ca2655253e71be546e1.49ec507c8c81787c05598347db28fe699ccea17f6f63e025aecca400cf4922f8
Model config BertConfig {
  "_name_or_path": "Geotrend/bert-base-pt-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,


...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.5854241338112306,
 'eval_f1': 0.5626841686842015,
 'eval_loss': 1.0454798936843872,
 'eval_precision': 0.5759576349042975,
 'eval_recall': 0.561198658157421,
 'eval_runtime': 1.6651,
 'eval_samples_per_second': 502.668,
 'eval_steps_per_second': 16.215}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 1.467  , -0.7935 ,  0.554  ,  0.287  , -2.082  ],
       [ 2.943  , -1.81   , -0.349  ,  1.024  , -2.637  ],
       [ 1.874  , -1.769  ,  2.326  , -0.831  , -1.694  ],
       ...,
       [ 2.568  , -2.459  ,  0.3594 ,  0.68   , -1.996  ],
       [-0.06494, -1.254  ,  3.256  , -0.1514 , -1.845  ],
       [ 2.613  , -1.986  ,  1.412  , -0.10223, -2.629  ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0ebe421d721676677f14dd2556297b76144988966ccb9ca2655253e71be546e1.49ec507c8c81787c05598347db28fe699ccea17f6f63e025aecca400cf4922f8
Model config BertConfig {
  "_name_or_path": "Geotrend/bert-base-pt-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",


  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0ebe421d721676677f14dd2556297b76144988966ccb9ca2655253e71be546e1.49ec507c8c81787c05598347db28fe699ccea17f6f63e025aecca400cf4922f8
Model config BertConfig {
  "_name_or_path": "Geotrend/bert-base-pt-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "L

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.5280764635603346,
 'eval_f1': 0.5362681377085261,
 'eval_loss': 1.0929874181747437,
 'eval_precision': 0.5013780373859207,
 'eval_recall': 0.6323572246768123,
 'eval_runtime': 1.6742,
 'eval_samples_per_second': 499.941,
 'eval_steps_per_second': 16.127}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 0.1782, -0.996 ,  2.299 , -0.2695, -1.591 ],
       [ 2.006 , -1.347 ,  1.023 ,  1.021 , -3.203 ],
       [ 1.732 , -1.834 ,  2.197 , -0.4438, -1.445 ],
       ...,
       [ 1.5625, -2.316 ,  1.687 ,  0.85  , -2.012 ],
       [ 0.1598, -1.537 ,  3.578 , -0.4612, -1.6875],
       [ 1.787 , -1.585 ,  2.174 ,  0.641 , -3.18  ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 2,
       0, 1, 0, 2, 3, 3,

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

### PT - [neuralmind/bert-base-portuguese-cased](https://huggingface.co/neuralmind/bert-base-portuguese-cased)

In [ ]:
model_name = "neuralmind/bert-base-portuguese-cased"

In [ ]:
train_model(model_name)

https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp82lewq6n


Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f1a9ba41d40e8c6f5ba4988aa2f7702c3b43768183e4b82483e04f2848841ecf.a6c00251b9344c189e2419373d6033016d0cd3d87ea59f6c86069046ac81956d
creating metadata file for /root/.cache/huggingface/transformers/f1a9ba41d40e8c6f5ba4988aa2f7702c3b43768183e4b82483e04f2848841ecf.a6c00251b9344c189e2419373d6033016d0cd3d87ea59f6c86069046ac81956d
https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp07j5k7k0


Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
creating metadata file for /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hi

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.6081242532855436,
 'eval_f1': 0.5935870215616074,
 'eval_loss': 0.9956874847412109,
 'eval_precision': 0.5856018554046334,
 'eval_recall': 0.6112540228004144,
 'eval_runtime': 1.6663,
 'eval_samples_per_second': 502.297,
 'eval_steps_per_second': 16.203}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 1.265e+00,  2.188e+00, -2.840e+00, -8.140e-01,  1.819e-01],
       [ 3.586e+00, -1.773e+00,  2.740e-02,  7.588e-01, -2.875e+00],
       [ 3.287e+00, -2.150e+00,  2.739e-03,  5.830e-01, -2.027e+00],
       ...,
       [ 3.029e+00, -2.287e+00,  8.232e-01,  1.601e+00, -3.428e+00],
       [ 5.996e-01, -2.311e+00,  3.338e+00,  5.654e-01, -2.354e+00],
       [ 1.851e+00, -2.674e+00,  2.270e+00,  1.332e+00, -3.076e+00]],
      dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
 

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_tran

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.5734767025089605,
 'eval_f1': 0.5786372051933095,
 'eval_loss': 0.9821927547454834,
 'eval_precision': 0.5407975495462425,
 'eval_recall': 0.6795055364642993,
 'eval_runtime': 1.7349,
 'eval_samples_per_second': 482.456,
 'eval_steps_per_second': 15.563}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 0.3708,  2.951 , -2.82  , -0.389 , -0.2441],
       [ 2.482 , -1.916 ,  1.772 ,  1.009 , -3.129 ],
       [ 2.662 , -1.683 ,  0.561 ,  0.906 , -2.344 ],
       ...,
       [ 2.463 , -2.158 ,  1.035 ,  1.905 , -3.15  ],
       [ 0.8296, -2.41  ,  3.56  ,  0.68  , -2.459 ],
       [ 1.517 , -2.594 ,  2.467 ,  1.728 , -2.885 ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 2,
       0, 1, 0, 2, 3, 3,

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Hyper

In [ ]:
train_model_hyper(model_name)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.084800,0.966433,0.594982,0.542655,0.622347,0.506537
2,0.764800,0.996963,0.597372,0.559421,0.584921,0.556668
3,0.608600,1.092008,0.608124,0.598246,0.584694,0.620953
4,0.500800,1.239289,0.592593,0.582328,0.576989,0.598212
5,0.422300,1.338160,0.579450,0.579239,0.564222,0.601224


...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32
Saving model checkpoint to ./results/run-4/checkpoint-942
Configuration saved in ./results/run-4/checkpoint-942/config.json
Model weights saved in ./results/run-4/checkpoint-942/pytorch_model.bin
tokenizer config file saved in ./results/run-4/checkpoint-942/tokenizer_config.json
Special tokens file saved in ./results/run-4/checkpoint-942/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 837
  Batch size = 32
Saving model checkpoint to ./results/run-4/checkpoint-1884
Configuration saved in ./results/run-4/checkpoint-1884/config.json
Model weights saved in ./results/run-4/checkpoint-1884/pytorch_model.bin
tokenizer config file saved in ./results/run-4/checkpoint-1884/tokenizer_config.json
Special tokens file saved in ./results/run-4/checkpoint-1884/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 837
  Batch size = 32
Saving model checkpoint to ./results/run-4/checkpoint-282

***** Hypertuning best run *****


BestRun(run_id='0', objective=0.5792388485370905, hyperparameters={'per_device_train_batch_size': 32, 'per_device_eval_batch_size': 32, 'weight_decay': 0.01, 'learning_rate': 2e-05, 'num_train_epochs': 5})

loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 

Epoch,Training Loss,Validation Loss


In [ ]:
!rm -rf ./results/

### PT - [neuralmind/bert-large-portuguese-cased](https://huggingface.co/neuralmind/bert-large-portuguese-cased)


In [ ]:
model_name = "neuralmind/bert-large-portuguese-cased"

In [ ]:
train_model(model_name)

https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpwolmyslh


Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/3a44fa9a74e90f509368a7f2789df38e1fedd153a52c62ef5cc5f4b0f5c99c2a.d61b68f744aef2741575c270d4ba0228cd35693bfa15d8babfb5c1079062d5d7
creating metadata file for /root/.cache/huggingface/transformers/3a44fa9a74e90f509368a7f2789df38e1fedd153a52c62ef5cc5f4b0f5c99c2a.d61b68f744aef2741575c270d4ba0228cd35693bfa15d8babfb5c1079062d5d7
https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpviajokxs


Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
creating metadata file for /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
loading configuration file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-large-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.6200716845878136,
 'eval_f1': 0.6040834571578513,
 'eval_loss': 0.9585959911346436,
 'eval_precision': 0.6181089324618736,
 'eval_recall': 0.6050553646429935,
 'eval_runtime': 4.3711,
 'eval_samples_per_second': 191.483,
 'eval_steps_per_second': 6.177}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 1.42   ,  1.717  , -2.535  , -0.5234 ,  0.01865],
       [ 3.045  , -1.982  , -0.3286 ,  1.792  , -3.297  ],
       [ 3.377  , -2.8    ,  0.811  ,  0.817  , -3.129  ],
       ...,
       [ 2.947  , -2.887  ,  0.612  ,  1.703  , -3.05   ],
       [ 1.689  , -3.295  ,  3.098  ,  0.6255 , -2.76   ],
       [ 2.512  , -3.197  ,  2.16   ,  1.627  , -3.541  ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-large-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_t

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-large-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
   

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.5961768219832736,
 'eval_f1': 0.6054071776170294,
 'eval_loss': 0.9971903562545776,
 'eval_precision': 0.5694053159109427,
 'eval_recall': 0.6830262368406699,
 'eval_runtime': 4.4413,
 'eval_samples_per_second': 188.456,
 'eval_steps_per_second': 6.079}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 0.8657,  3.812 , -2.768 , -0.3152, -1.829 ],
       [ 3.021 , -2.182 , -0.3164,  2.09  , -3.418 ],
       [ 3.008 , -3.111 ,  1.633 ,  1.261 , -3.318 ],
       ...,
       [ 2.314 , -3.12  ,  1.641 ,  1.635 , -3.033 ],
       [ 0.6284, -3.104 ,  4.16  ,  1.027 , -2.602 ],
       [ 1.993 , -3.428 ,  3.127 ,  1.451 , -3.361 ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 2,
       0, 1, 0, 2, 3, 3,

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

### PT - [monilouise/ner_news_portuguese ](https://huggingface.co/monilouise/ner_news_portuguese)

In [ ]:
model_name = "monilouise/ner_news_portuguese"

In [ ]:
train_model(model_name)

Downloading:   0%|          | 0.00/529 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.6069295101553166,
 'eval_f1': 0.5998235101021614,
 'eval_loss': 0.9637839198112488,
 'eval_precision': 0.6127578604216987,
 'eval_recall': 0.600057819233077,
 'eval_runtime': 1.6364,
 'eval_samples_per_second': 511.49,
 'eval_steps_per_second': 16.5}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 1.531  ,  2.014  , -2.588  , -0.5044 , -0.635  ],
       [ 3.25   , -1.405  ,  0.09534,  0.683  , -2.826  ],
       [ 3.238  , -1.967  ,  0.798  ,  0.3672 , -2.3    ],
       ...,
       [ 3.166  , -1.813  ,  0.238  ,  1.151  , -2.86   ],
       [ 0.7837 , -2.408  ,  2.787  ,  0.5234 , -2.393  ],
       [ 2.486  , -2.3    ,  1.229  ,  1.451  , -3.18   ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/monilouise/ner_news_portuguese
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/monilouise/ner_news_portuguese/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/monilouise/ner_news_portuguese/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/monilouise/ner_news_portuguese/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/monilouise/ner_news_portuguese/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/monilouise/ner_news_portuguese/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ee9b506d42deb9a1fde19bb7f40f64da74a2b871a58bfbfa55ccf29281aa9476.08868e7914995036411ed3728b7e905ee94d69938262ad30ffe4e5dac473f090
Model config BertConfig {
  "_name_or_path": "monilouise/ner_news_portuguese",
  "_num_labels": 13,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "L-ORG",
    "1": "O",
    "2": "I-PESSOA",
    "3": "I-ORG",
    "4": "B-PUB",
    "5": "L-PUB",
    "6": "I-LOC",
    "7": "B-PESSOA",
    "8": "L-LOC",
    "9": "B-LOC",
    "10": "B-ORG",
    "11": "L-PESSOA",
    "12": "I-PUB"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LO

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/monilouise/ner_news_portuguese/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ee9b506d42deb9a1fde19bb7f40f64da74a2b871a58bfbfa55ccf29281aa9476.08868e7914995036411ed3728b7e905ee94d69938262ad30ffe4e5dac473f090
Model config BertConfig {
  "_name_or_path": "monilouise/ner_news_portuguese",
  "_num_labels": 13,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LAB

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.5710872162485066,
 'eval_f1': 0.5765075735510389,
 'eval_loss': 1.0004297494888306,
 'eval_precision': 0.5387391221172878,
 'eval_recall': 0.6739892543500792,
 'eval_runtime': 1.6434,
 'eval_samples_per_second': 509.322,
 'eval_steps_per_second': 16.43}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 0.4407,  3.342 , -2.875 ,  0.4666, -1.863 ],
       [ 2.213 , -2.145 ,  2.215 ,  0.8574, -3.184 ],
       [ 2.713 , -1.936 ,  0.7695,  1.063 , -2.559 ],
       ...,
       [ 2.426 , -1.882 ,  0.551 ,  1.99  , -2.928 ],
       [ 0.7124, -2.414 ,  3.38  ,  0.737 , -2.824 ],
       [ 1.394 , -2.662 ,  2.398 ,  1.911 , -3.217 ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 2,
       0, 1, 0, 2, 3, 3,

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/monilouise/ner_news_portuguese
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/monilouise/ner_news_portuguese/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/monilouise/ner_news_portuguese/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/monilouise/ner_news_portuguese/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/monilouise/ner_news_portuguese/special_tokens_map.json


In [ ]:
!rm -rf ./results/

###  Multi - [bert-base-multilingual-cased](https://huggingface.co/bert-base-multilingual-cased)

In [ ]:
model_name = "bert-base-multilingual-cased"

In [ ]:
train_model(model_name)

https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp2al83lmm


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpfchbbi08


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
creating metadata file for /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidde

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.5746714456391876,
 'eval_f1': 0.5429565515611482,
 'eval_loss': 1.0504605770111084,
 'eval_precision': 0.5516580674799236,
 'eval_recall': 0.5434216985763377,
 'eval_runtime': 1.6511,
 'eval_samples_per_second': 506.935,
 'eval_steps_per_second': 16.353}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 1.732 , -0.18  ,  0.839 ,  1.183 , -2.44  ],
       [ 3.217 , -0.954 ,  0.1229,  0.5415, -2.51  ],
       [ 2.777 , -1.399 ,  1.224 , -0.721 , -1.147 ],
       ...,
       [ 2.66  , -1.473 , -0.0625,  1.548 , -2.26  ],
       [ 0.4626, -1.438 ,  3.56  , -0.3323, -1.701 ],
       [ 2.79  , -1.834 ,  1.816 ,  0.645 , -2.875 ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 2,
       0, 1, 0, 2, 3, 3,

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "abs

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },


...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.5173237753882916,
 'eval_f1': 0.518776468272993,
 'eval_loss': 1.1104624271392822,
 'eval_precision': 0.4849293392693143,
 'eval_recall': 0.6057699230895107,
 'eval_runtime': 1.6164,
 'eval_samples_per_second': 517.819,
 'eval_steps_per_second': 16.704}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 0.761  ,  0.0902 ,  1.144  , -0.639  , -1.148  ],
       [ 2.217  ,  0.5137 , -0.593  ,  0.8994 , -2.08   ],
       [ 1.788  , -1.356  ,  1.9375 ,  0.1042 , -2.115  ],
       ...,
       [ 0.5757 , -1.904  ,  2.857  ,  0.9937 , -2.287  ],
       [ 0.3494 , -1.272  ,  3.484  ,  0.01202, -1.701  ],
       [ 1.732  , -1.469  ,  1.455  ,  1.344  , -2.695  ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

###  Multi - [bert-base-multilingual-uncased](https://huggingface.co/bert-base-multilingual-uncased)

In [ ]:
model_name = "bert-base-multilingual-uncased"

In [ ]:
train_model(model_name)

https://huggingface.co/bert-base-multilingual-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp1pl8vt4i


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/1b935b135ddb021a7d836c00f5702b80d11d348fd5c5a42cbd933d8ed1f55be9.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/1b935b135ddb021a7d836c00f5702b80d11d348fd5c5a42cbd933d8ed1f55be9.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpodolx7gr


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
creating metadata file for /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
loading configuration file https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.6021505376344086,
 'eval_f1': 0.5781114357419681,
 'eval_loss': 1.0308383703231812,
 'eval_precision': 0.5886512827982993,
 'eval_recall': 0.5768592156221023,
 'eval_runtime': 1.6019,
 'eval_samples_per_second': 522.516,
 'eval_steps_per_second': 16.855}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 1.361  ,  1.08   , -1.794  , -1.087  ,  0.084  ],
       [ 3.03   , -1.139  , -0.4194 , -0.02048, -2.172  ],
       [ 1.711  , -1.693  ,  2.76   , -0.1273 , -2.516  ],
       ...,
       [ 2.207  , -2.424  ,  1.112  ,  0.5923 , -2.213  ],
       [ 0.6587 , -1.643  ,  2.96   ,  0.10114, -2.295  ],
       [ 2.465  , -1.575  ,  0.5103 ,  1.292  , -3.406  ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": 

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
 

...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.5280764635603346,
 'eval_f1': 0.5324557256432719,
 'eval_loss': 1.0950160026550293,
 'eval_precision': 0.4984722162652903,
 'eval_recall': 0.6155574646811761,
 'eval_runtime': 1.6271,
 'eval_samples_per_second': 514.417,
 'eval_steps_per_second': 16.594}

***** Running Prediction *****
  Num examples = 838
  Batch size = 32


PredictionOutput(predictions=array([[ 0.8643,  2.037 , -0.562 , -0.3137, -1.917 ],
       [ 1.85  , -1.251 ,  0.8174, -0.4863, -1.512 ],
       [ 0.86  , -1.485 ,  3.195 ,  0.1565, -2.578 ],
       ...,
       [ 1.356 , -2.24  ,  1.709 ,  0.4204, -1.677 ],
       [ 0.279 , -1.377 ,  3.473 ,  0.1666, -2.432 ],
       [ 1.355 , -1.665 ,  1.236 ,  1.179 , -2.604 ]], dtype=float16), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 3, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 2,
       0, 1, 0, 2, 3, 3,

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased/special_tokens_map.json


In [ ]:
!rm -rf ./results/